In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, GPT2LMHeadModel
from tqdm.notebook import tqdm
import numpy as np
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')


In [ ]:
!nvidia-smi

In [ ]:
input_ids = tf.constant(tokenizer.encode("i believe i can fly, i believe", add_special_tokens=True))[None, :] # Batch size 1


In [ ]:
outputs = model(input_ids)

In [ ]:
logits = outputs[0]

In [ ]:
logits.shape

In [ ]:
import numpy as np
tokenizer.decode(np.argmax(np.array(logits[0]), axis = 1))

In [ ]:
model.summary()

In [ ]:
max_length = 64
inputs_ids = Input(shape = (max_length,), dtype = 'int32')

In [ ]:
tokenizer.encode_plus('START my bird is beautiful', 'but a bit ugly', add_special_tokens = True)

In [ ]:
tokenizer.decode([616])

In [ ]:
def generate(input_sentence, max_generation = 100, repetition_penalty = 1.0):
    
    input_ids = tokenizer.encode(input_sentence, add_special_tokens=True)
    for i in tqdm(range(max_generation)):
        inp = tf.constant(input_ids)[None, :]
#         if i == 0:
        outputs = model(inp)
#         else:
#             outputs = model(inp, past = past)
        logits = np.array(outputs[0][0])
        past = outputs[1]
        
        if repetition_penalty != 1:
            for elt in input_ids:
                if logits[-1,elt] < 0:
                    logits[-1,elt] *= repetition_penalty
                else:
                    logits[-1,elt] /= repetition_penalty
        
        res = np.argmax(np.array(logits), axis = 1)
        input_ids.append(res[-1])
        
    print(tokenizer.decode(input_ids))
    return input_ids

In [ ]:
input_ids = generate('i am a bird', repetition_penalty = 1.5)

In [ ]:
for elt in input_ids:
    print(tokenizer.decode([elt]), elt)

In [ ]:
input_ids

In [ ]:
dir(model)